# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.02250


extracting tarball to tmp_2301.02250...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.02252


extracting tarball to tmp_2301.02252...

...

/tmp/ipykernel_2116/4030337529.py:34: LatexWarning: 2301.02559 did not run properly
Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table

  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2301.02584


extracting tarball to tmp_2301.02584...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.02652


extracting tarball to tmp_2301.02652... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02265-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02265) | **The Pristine survey -- XX: GTC follow-up observations of extremely  metal-poor stars identified from Pristine and LAMOST**  |
|| Anke Arentsen, et al. -- incl., <mark>Jonay I. González Hernández</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *13 pages, accepted for publication in MNRAS*|
|**Abstract**| Ultra metal-poor stars ([Fe/H] < -4.0) are very rare, and finding them is a challenging task. Both narrow-band photometry and low-resolution spectroscopy have been useful tools for identifying candidates, and in this work we combine both approaches. We cross-matched metallicity-sensitive photometry from the Pristine survey with the low-resolution spectroscopic LAMOST database, and re-analysed all LAMOST spectra with [Fe/H]_Pristine < -2.5. We find that ~1/3rd of this sample (selected without [Fe/H]_Pristine quality cuts) also have spectroscopic [Fe/H] < -2.5. From this sample, containing many low signal-to-noise (S/N) spectra, we selected eleven stars potentially having [Fe/H] < -4.0 or [Fe/H] < -3.0 with very high carbon abundances, and we performed higher S/N medium-resolution spectroscopic follow-up with OSIRIS on the 10.4m Gran Telescopio Canarias (GTC). We confirm their extremely low metallicities, with a mean of [Fe/H] = -3.4 and the most metal-poor star having [Fe/H]= -3.8. Three of these are clearly carbon-enhanced metal-poor (CEMP) stars with +1.65 < [C/Fe] < +2.45. The two most carbon-rich stars are either among the most metal-poor CEMP-s stars or the most carbon-rich CEMP-no stars known, the third is likely a CEMP-no star. We derived orbital properties for the OSIRIS sample and find that only one of our targets can be confidently associated with known substructures/accretion events, and that three out of four inner halo stars have prograde orbits. Large spectroscopic surveys may contain many hidden extremely and ultra metal-poor stars, and adding additional information from e.g. photometry as in this work can uncover them more efficiently and confidently. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02313-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02313) | **MIRI/JWST observations reveal an extremely obscured starburst in the  z=6.9 system SPT0311-58**  |
|| J. Álvarez-Márquez, et al. -- incl., <mark>F. Walter</mark>, <mark>A. Alonso-Herrero</mark>, <mark>M. Güdel</mark>, <mark>Th. Henning</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *18 pages, 8 figures, acepted for publication in A&A*|
|**Abstract**| Using MIRI on-board JWST we present mid-infrared sub-arcsec imaging (MIRIM) and spectroscopy (MRS) of the hyperluminous infrared system SPT0311-58 at z=6.9. MIRI observations are compared with existing ALMA far-infrared continuum and [CII]158$\mu$m imaging. Even though the ALMA observations suggests very high star formation rates (SFR) in the eastern (E) and western (W) galaxies of the system, the H$\alpha$ line is not detected. This, together with the detection of the Pa$\alpha$ line, implies very high optical nebular extinction with lower limits of 4.2 (E) and 3.9 mag (W), and even larger 5.6 (E) and 10.0 (W) for SED derived values. The extinction-corrected Pa$\alpha$ SFRs are 383 and 230M$_{\odot}$yr$^{-1}$ for the E and W galaxies, respectively. This represents 50% of the SFRs derived from the [CII]158$\mu$m line and infrared light for the E galaxy and as low as 6% for the W galaxy. The specific SFR in the stellar clumps ranges from 25 to 59Gyr$^{-1}$, which are 3 to 10 times larger than the values measured in galaxies of similar mass at redshifts 6 to 8. The MIRI observations reveal a clumpy stellar structure, with each clump having 3 to 5 $\times$10$^{9}$M$_\mathrm{\odot}$, leading to a total stellar mass of 2.0 and 1.5$\times$10$^{10}$M$_\mathrm{\odot}$ for the E and W galaxies, respectively. The overall gas mass fraction is $M_\mathrm{gas}$/$M_*\sim3$, similar to that of z=4.5-6 star-forming galaxies. The observed properties of SPT0311-58 such as the clumpy distribution at sub(kpc) scales and the very high average extinction are similar to those observed in low- and intermediate-z LIRGs and ULIRGs, even though SPT0311-58 is observed only 800 Myr after the Big Bang. Massive, heavily obscured, clumpy starburst systems like SPT0311-58 likely represent the early phases in the formation of massive high-z bulge/spheroids and luminous quasars. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02477-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02477) | **The CARMENES search for exoplanets around M dwarfs, Wolf 1069 b:  Earth-mass planet in the habitable zone of a nearby, very low-mass star**  |
|| <mark>D. Kossakowski</mark>, et al. -- incl., <mark>M. Kürster</mark>, <mark>T. Trifonov</mark>, <mark>Th. Henning</mark>, <mark>R. Burn</mark>, <mark>E. Herrero</mark>, <mark>A. Pavlov</mark>, <mark>M. Pérez-Torres</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *26 pages, 15 figures*|
|**Abstract**| We present the discovery of an Earth-mass planet ($M_b\sin i = 1.36\pm0.21M_\oplus$) on a 15.6d orbit of a relatively nearby ($d\sim$9.6pc) and low-mass ($0.167\pm0.011 M_\odot$) M5.0V star, Wolf 1069. Sitting at a separation of $0.0672\pm0.0014$au away from the host star puts Wolf 1069b in the habitable zone (HZ), receiving an incident flux of $S=0.652\pm0.029S_\oplus$. The planetary signal was detected using telluric-corrected radial-velocity (RV) data from the CARMENES spectrograph, amounting to a total of 262 spectroscopic observations covering almost four years. There are additional long-period signals in the RVs, one of which we attribute to the stellar rotation period. This is possible thanks to our photometric analysis including new, well-sampled monitoring campaigns undergone with the OSN and TJO facilities that supplement archival photometry (i.e., from MEarth and SuperWASP), and this yielded an updated rotational period range of $P_{rot}=150-170$d, with a likely value at $169.3^{+3.7}_{-3.6}$d. The stellar activity indicators provided by the CARMENES spectra likewise demonstrate evidence for the slow rotation period, though not as accurately due to possible factors such as signal aliasing or spot evolution. Our detectability limits indicate that additional planets more massive than one Earth mass with orbital periods of less than 10 days can be ruled out, suggesting that perhaps Wolf 1069 b had a violent formation history. This planet is also the 6th closest Earth-mass planet situated in the conservative HZ, after Proxima Centauri b, GJ 1061d, Teegarden's Star c, and GJ 1002 b and c. Despite not transiting, Wolf 1069b is nonetheless a very promising target for future three-dimensional climate models to investigate various habitability cases as well as for sub-ms$^{-1}$ RV campaigns to search for potential inner sub-Earth-mass planets in order to test planet formation theories. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02558-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02558) | **General relativistic effects and the near-infrared and X-ray variability  of Sgr A* I**  |
|| Sebastiano D. von Fellenberg, et al. -- incl., <mark>Frank Eisenhauer</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| **|
|**Abstract**| The near-infrared (NIR) and X-ray emission of Sagittarius A* shows occasional bright flares that are assumed to originate from the innermost region of the accretion flow. We identified $25$ $4.5 \mu m$ and $24$ X-ray flares in archival data obtained with the \textit{Spitzer} and \textit{Chandra} observatories. With the help of general relativistic ray-tracing code, we modeled trajectories of ``hot spots'' and studied the light curves of the flares for signs of the effects of general relativity. Despite their apparent diversity in shape, all flares share a common, exponential impulse response, a characteristic shape that is the building block of the variability. This shape is symmetric, that is, the rise and fall times are the same. Furthermore, the impulse responses in the NIR and X-ray are identical within uncertainties, with an exponential time constant $\tau\sim 15$ minute. The observed characteristic flare shape is inconsistent with hot-spot orbits viewed edge-on. Individually modeling the light curves of the flares, we derived constraints on the inclination of the orbital plane of the hot spots with respect to the observer ($i \sim 30^{\circ} , < 75^{\circ} $) and on the characteristic timescale of the intrinsic variability (tens of minutes). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02584-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02584) | **The Bright Extragalactic ALMA Redshift Survey (BEARS) II: Millimetre  photometry of gravitational lens candidates**  |
|| G. J. Bendo, et al. -- incl., <mark>A. I. Harris</mark>, <mark>C. N. Herrera</mark>, <mark>D. H. Hughes</mark>, <mark>M. W. L. Smith</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *Accepted for publication in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| We present 101 and 151 GHz ALMA continuum images for 85 fields selected from Herschel observations that have 500 micron flux densities >80 mJy and 250-500 micron colours consistent with z > 2, most of which are expected to be gravitationally lensed or hyperluminous infrared galaxies. Approximately half of the Herschel 500 micron sources were resolved into multiple ALMA sources, but 11 of the 15 brightest 500 micron Herschel sources correspond to individual ALMA sources. For the 37 fields containing either a single source with a spectroscopic redshift or two sources with the same spectroscopic redshift, we examined the colour temperatures and dust emissivity indices. The colour temperatures only vary weakly with redshift and are statistically consistent with no redshift-dependent temperature variations, which generally corresponds to results from other samples selected in far-infrared, submillimetre, or millimetre bands but not to results from samples selected in optical or near-infrared bands. The dust emissivity indices, with very few exceptions, are largely consistent with a value of 2. We also compared spectroscopic redshifts to photometric redshifts based on spectral energy distribution templates designed for infrared-bright high-redshift galaxies. While the templates systematically underestimate the redshifts by ~15%, the inclusion of ALMA data decreases the scatter in the predicted redshifts by a factor of ~2, illustrating the potential usefulness of these millimetre data for estimating photometric redshifts. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02652-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02652) | **Diverse Carbonates in Exoplanet Oceans Promote the Carbon Cycle**  |
|| Kaustubh Hakim, Meng Tian, Dan J. Bower, <mark>Kevin Heng</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *Published in ApJ Letters*|
|**Abstract**| Carbonate precipitation in oceans is essential for the carbonate-silicate cycle (inorganic carbon cycle) to maintain temperate climates. By considering the thermodynamics of carbonate chemistry, we demonstrate that the ocean pH decreases by approximately 0.5 for a factor of 10 increase in the atmospheric carbon dioxide content. The upper and lower limits of ocean pH are within 1-4 of each other, where the upper limit is buffered by carbonate precipitation and defines the ocean pH when the carbon cycle operates. If the carbonate compensation depth (CCD) resides above the ocean floor, then carbonate precipitation and the carbon cycle cease to operate. The CCD is deep (>40 km) for high ocean temperature and high atmospheric carbon dioxide content. Key divalent carbonates of magnesium, calcium and iron produce an increasingly wider parameter space of deep CCDs, suggesting that chemical diversity promotes the carbon cycle. The search for life from exoplanets will benefit by including chemically more diverse targets than Earth twins. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02338-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02338) | **GOALS-JWST: Pulling Back the Curtain on the AGN and Star Formation in VV  114**  |
|| J. Rich, et al. -- incl., <mark>S. Linden</mark>, <mark>M. J. I. Brown</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *12 pages, 4 figures, accepted for publication in ApJL*|
|**Abstract**| We present results from the James Webb Space Telescope (JWST) Director's Discretionary Time Early Release Science (ERS) program 1328 targeting the nearby, Luminous Infrared Galaxy (LIRG), VV 114. We use the MIRI and NIRSpec instruments to obtain integral-field spectroscopy of the heavily obscured Eastern nucleus (V114E) and surrounding regions. The spatially resolved, high-resolution, spectra reveal the physical conditions in the gas and dust over a projected area of 2-3 kpc that includes the two brightest IR sources, the NE and SW cores. Our observations show for the first time spectroscopic evidence that the SW core hosts an AGN as evidenced by its very low 6.2 {\mu}m and 3.3 {\mu}m PAH equivalent widths (0.12 and 0.017 {\mu}m respectively) and mid and near-IR colors. Our observations of the NE core show signs of deeply embedded star formation including absorption features due to aliphatic hydrocarbons, large quantities of amorphous silicates, as well as HCN due to cool gas along the line of sight. We detect elevated [Fe II]/Pf{\alpha} consistent with extended shocks coincident with enhanced emission from warm H$_{2}$, far from the IR-bright cores and clumps. We also identify broadening and multiple kinematic components in both H$_{2}$ and fine structure lines caused by outflows and previously identified tidal features. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xd0 in position 8633: invalid continuation byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.02559-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.02559) | **Forbidden emission lines in protostellar outflows and jets with MUSE**  |
|| <mark>Lizxandra Flores-Rivera</mark>, et al. -- incl., <mark>Mario Flock</mark>, <mark>Nicolás Kurtovic</mark>, <mark>André Müller</mark>, <mark>Christian Fendt</mark>, <mark>Thomas Henning</mark>, <mark>Roy van Boekel</mark>, <mark>Ralf Launhardt</mark> |
|*Appeared on*| *2023-01-09*|
|*Comments*| *Accepted on December 19 2022*|
|**Abstract**| Forbidden emission lines in protoplanetary disks are a key diagnostic in studies of the evolution of the disk and the host star. We report spatially resolved emission lines, [OI] 6300, 6363, [NII] 6548, 6583, H$\alpha$, and [SII] 6716, 6730 Angstrom that are believed to be associated with jets and magnetically driven winds in the inner disks. Observations were carried out with the optical integral field spectrograph of the Multi Unit Spectroscopic Explorer (MUSE), at the Very Large Telescope (VLT). With a resolution of 0.025 X 0.025 arcsec$^{2}$, we aim to derive the position angle of the outflow/jet (PA$_{outflow/jet}$) that is connected with the inner disk. The forbidden emission lines analyzed here have their origin at the inner parts of the protoplanetary disk. From the maximum intensity emission along the outflow/jet in DL Tau, CI Tau, DS Tau, IP Tau, and IM Lup, we were able to reliably measure the PA$_{outflow/jet}$ for most of the identified lines. We found that our estimates agree with PA$_{dust}$ for most of the disks. These estimates depend on the signal-to-noise level and the collimation of the outflow (jet). The outflows/jets in CIDA 9, GO Tau, and GW Lup are too compact for a PA$_{outflow/jet}$ to be estimated. Based on our kinematics analysis, we confirm that DL Tau and CI Tau host a strong outflow/jet with line-of-sight velocities much greater than 100 km s$^{-1}$, whereas DS Tau, IP Tau, and IM Lup velocities are lower and their structures encompass low-velocity components to be more associated with winds. Our estimates for the mass-loss rate, $\dot{M}_{{loss}}$, range between (1.1-6.5)X10$^{-7}$-10$^{-8}$ $M_{\odot}$ yr$^{-1}$ for the disk-outflow/jet systems analyzed here. The outflow/jet systems analyzed here are aligned within around 1 degree between the inner and outer disk. Further observations are needed to confirm a potential misalignment in IM Lup. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table
</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.02265.md
    + _build/html/tmp_2301.02265/./figures/osiris_prist_4-eps-converted-to.png
    + _build/html/tmp_2301.02265/./figures/kiel-ulyss.png
    + _build/html/tmp_2301.02265/./figures/teff-logg_FERRE-adopted_new.png
exported in  _build/html/2301.02313.md
    + _build/html/tmp_2301.02313/./plot_apertures.png
    + _build/html/tmp_2301.02313/./plot_MRSapert.png
    + _build/html/tmp_2301.02313/./plot_fracgas_vs_z.png
exported in  _build/html/2301.02477.md
    + _build/html/tmp_2301.02477/./20221205_wolf1069_periodogram_carmvis_rv_tellcorr_2cols_equalzoom.png
    + _build/html/tmp_2301.02477/./20221205_wolf1069_periodogram_carmvis_tellcorr_2121_bluered_2cols_zoom_martinfap.png
    + _build/html/tmp_2301.02477/./rv_vs_time.png
    + _build/html/tmp_2301.02477/./phased_rv_pl1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\noteab}[1]{\hl{[AA: #1]}}$
$\newcommand{\rev}[1]{{\textbf{\color{magenta}#1}}}$
$\newcommand{\teff}{\ensuremath{T_{\mathrm{eff}}}\xspace}$
$\newcommand{\teffa}{\ensuremath{T_{\mathrm{eff,A}}}\xspace}$
$\newcommand{\kms}{\ensuremath{\rm{km} s^{-1}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\feh}{\rm{[Fe/H]}\xspace}$
$\newcommand{\cfe}{\rm{[C/Fe]}\xspace}$
$\newcommand{\nfe}{\rm{[N/Fe]}\xspace}$
$\newcommand{\ch}{\rm{[C/H]}\xspace}$
$\newcommand{\ac}{\rm{A(C)}\xspace}$
$\newcommand{\mgfe}{\rm{[Mg/Fe]}\xspace}$
$\newcommand{\alphafe}{\rm{[\ensuremath{\alpha}/Fe]}\xspace}$
$\newcommand{\Gaia}{\textit{Gaia}\xspace}$
$\newcommand{\CaHK}{\emph{CaHK}\xspace}$
$\newcommand{\Pristine}{\emph{Pristine}\xspace}$
$\newcommand{\FERRE}{{\tt FERRE}\xspace}$
$\newcommand{\ULySS}{{\tt ULySS}\xspace}$
$\newcommand{\AA}{\normalfont\r{A}\xspace}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{\msun}{{\rm M_\odot}}$
$\newcommand{\eg}{{e.g., }}$
$\newcommand{\ie}{{i.e., }}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\noteab$}[1]{$\hl${[AA: #1]}}$
$\newcommand{$\rev$}[1]{{\textbf{\color{magenta}#1}}}$
$\newcommand{$\teff$}{$\ensuremath${T_{\mathrm{eff}}}$\xspace$}$
$\newcommand{$\teff$a}{$\ensuremath${T_{\mathrm{eff,A}}}$\xspace$}$
$\newcommand{$\kms$}{$\ensuremath${\rm{km} s^{-1}}$\xspace$}$
$\newcommand{$\logg$}{$\ensuremath${\log g}$\xspace$}$
$\newcommand{$\feh$}{\rm{[Fe/H]}$\xspace$}$
$\newcommand{$\cfe$}{\rm{[C/Fe]}$\xspace$}$
$\newcommand{$\nfe$}{\rm{[N/Fe]}$\xspace$}$
$\newcommand{$\ch$}{\rm{[C/H]}$\xspace$}$
$\newcommand{$\ac$}{\rm{A(C)}$\xspace$}$
$\newcommand{$\mgfe$}{\rm{[Mg/Fe]}$\xspace$}$
$\newcommand{$\alphafe$}{\rm{[$\ensuremath${\alpha}/Fe]}$\xspace$}$
$\newcommand{$\Gaia$}{\textit{Gaia}$\xspace$}$
$\newcommand{$\CaHK$}{\emph{CaHK}$\xspace$}$
$\newcommand{$\Pristine$}{\emph{Pristine}$\xspace$}$
$\newcommand{$\FERRE$}{{\tt FERRE}$\xspace$}$
$\newcommand{$\ULySS$}{{\tt ULySS}$\xspace$}$
$\newcommand{$\AA$}{\normalfont\r{A}$\xspace$}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\newcommand{$\msun$}{{\rm M_\odot}}$
$\newcommand{$\eg$}{{e.g., }}$
$\newcommand{$\ie$}{{i.e., }}$</div>



<div id="title">

# The Pristine survey -- XX: GTC follow-up observations of extremely metal-poor stars identified from Pristine and LAMOST

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.02265-b31b1b.svg)](https://arxiv.org/abs/2301.02265)<mark>Appeared on: 2023-01-09</mark> - _13 pages, accepted for publication in MNRAS_

</div>
<div id="authors">

Anke Arentsen, et al. -- incl., <mark><mark>Jonay I. González Hernández</mark></mark>

</div>
<div id="abstract">

**Abstract:** Ultra metal-poor stars ($\feh < -4.0$) are very rare, and finding them is a challenging task. Both narrow-band photometry and low-resolution spectroscopy have been useful tools for identifying candidates, and in this work we combine both approaches.We cross-matched metallicity-sensitive photometry from the$\Pristine$survey with the low-resolution spectroscopic LAMOST database, and re-analysed all LAMOST spectra with$\feh_$\Pristine$<-2.5$. We find that$\sim$1/3rd of this sample (selected without$\feh_$\Pristine$$quality cuts) also have spectroscopic$\feh < -2.5$.From this sample, containing many low signal-to-noise (S/N) spectra, we selected eleven stars potentially having$\feh < -4.0$or$\feh < -3.0$with very high carbon abundances, and we performed higher S/N medium-resolution spectroscopic follow-up with OSIRIS on the 10.4m Gran Telescopio Canarias (GTC). We confirm their extremely low metallicities, with a mean of$\feh = -3.4$and the most metal-poor star having$\feh = -3.8$.Three of these are clearly carbon-enhanced metal-poor (CEMP) stars with$+1.65 < $\cfe$ < +2.45$.The two most carbon-rich stars are either among the most metal-poor CEMP-s stars or the most carbon-rich CEMP-no stars known, the third is likely a CEMP-no star.We derived orbital properties for the OSIRIS sample and find that only one of our targets can be confidently associated with known substructures/accretion events, and that three out of four inner halo stars have prograde orbits.Large spectroscopic surveys may contain many hidden extremely and ultra metal-poor stars, and adding additional information from e.g. photometry as in this work can uncover them more efficiently and confidently.

</div>

<div id="div_fig1">

<img src="tmp_2301.02265/./figures/osiris_prist_4-eps-converted-to.png" alt="Fig5" width="100%"/>

**Figure 5. -** OSIRIS/GTC spectra (3750 $\AA$-4500 $\AA$) of our stellar sample
(black line) and the best fits calculated with $\FERRE$, colour-coded by $\teff$(the bluer the hotter) and sorted by decreasing $\feh$. The Balmer lines (yellow) and main metallic absorption features (purple) are high-lighted. Above each spectrum the metallicity, effective temperature and carbonicity are displayed.  (*fig:osiris*)

</div>
<div id="div_fig2">

<img src="tmp_2301.02265/./figures/kiel-ulyss.png" alt="Fig1" width="100%"/>

**Figure 1. -** Top: Kiel diagram for all exposures of the 4900 unique $\Pristine$-selected stars in LAMOST analysed with $\ULySS$, colour-coded by metallicity. No quality cuts were applied to the photometric metallicities in the selection. The results for the eleven stars that were followed up with OSIRIS (see Section \ref{sec:osiris}) are highlighted with larger symbols (the two high and low $\logg$ outliers are CEMP stars). Bottom: $\ULySS$ metallicity histogram of the same sample in black, and $\FERRE$ metallicity histogram for the VMP sub-sample in red.  (*fig:ulyss*)

</div>
<div id="div_fig3">

<img src="tmp_2301.02265/./figures/teff-logg_FERRE-adopted_new.png" alt="Fig2" width="100%"/>

**Figure 2. -** Kiel diagram showing the pure FERRE stellar parameters (black) and the adopted stellar parameters and uncertainties (magenta). See the text for details. Also shown are Yonsei-Yale isochrones for two different metallicities (both with age = 12 Gyr, $\alphafe = +0.4$). (*fig:tefflogg*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\arcs}{\arcsec\xspace}$
$\newcommand{\CII}{[\ion{C}{II}]158\mum}$
$\newcommand{\oddpm}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \-#2}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\arcs$}{$\arcsec$$\xspace$}$
$\newcommand{$\CII$}{[$\ion${C}{II}]158\mum}$
$\newcommand{$\oddpm$}[2]{\raisebox{0.5ex}{\tiny\substack{+#1 \-#2}}}$</div>



<div id="title">

# MIRI/JWST observations reveal an extremely obscured starburst in the z=6.9 system SPT0311-58

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.02313-b31b1b.svg)](https://arxiv.org/abs/2301.02313)<mark>Appeared on: 2023-01-09</mark> - _18 pages, 8 figures, acepted for publication in A&A_

</div>
<div id="authors">

J. Álvarez-Márquez, et al. -- incl., <mark><mark>F. Walter</mark></mark>, <mark><mark>A. Alonso-Herrero</mark></mark>, <mark><mark>M. Güdel</mark></mark>, <mark><mark>Th. Henning</mark></mark>

</div>
<div id="abstract">

**Abstract:** Luminous infrared starbursts in the early universe are thought to be the progenitors of massive quiescent galaxies identified at redshifts 2 to 4. Using MIRI on-board JWST we present mid-infrared sub-arcsec imaging (MIRIM) and spectroscopy (MRS) of such a starburst: the slightly lensed hyperluminous infrared system SPT0311-58 at$z$=6.9. Our observations target the stellar (rest-frame 1.26$\mu$m emission) structure and ionised (Pa$\alpha$and H$\alpha$) medium on kpc scales in the system. MIRI observations are compared with existing ALMA far-infrared continuum and[$\ion${C}{II}]158$\mu$m imaging at similar angular resolution. Even though the ALMA observations imply very high star formation rates in the eastern (E) and western (W) galaxies of the system, the H$\alpha$line is, strikingly, not detected in our MRS observations. This, together with the detection of the ionised gas phase in Pa$\alpha$, implies very high internal nebular extinction with lower limits ($A_\mathrm{V}$) of 4.2 (E) and 3.9 mag (W), and even larger (5.6 (E) and 10.0 (W)) for SED derived values. The extinction-corrected Pa$\alpha$lower limits to the star formation rates are 383 and 230 M$_\mathrm{\odot}$yr$^{-1}$for the E and W galaxies, respectively. This represents 50\%of the SFRs derived from the[$\ion${C}{II}]158$\mu$m line and infrared light for the E galaxy and as low as 6\%for the W galaxy. The specific star formation (sSFR) in the stellar clumps ranges from 25 to 59 Gyr$^{-1}$assuming a star formation with a 50-100 Myr constant rate. This sSFR is 3 to 10 times larger than the values measured in galaxies of similar mass at redshifts 6 to 8: SPT0311-58 thus clearly stands out as a starburst system when compared with typical massive star-forming galaxies at similar high redshifts. The MIRI observations reveal a clumpy stellar structure, with each clump having  3 to 5$\times$10$^{9}$M$_\mathrm{\odot}$mass in stars, leading to a total stellar mass of 2.0 and 1.5$\times$10$^{10}$M$_\mathrm{\odot}$for the E and W galaxies, respectively. The overall gas mass fraction is$M_\mathrm{gas}$/$M_*\sim 3$, similar to that of$z$$\sim$4.5-6 star-forming galaxies, suggesting a flattening of the gas mass fraction in massive starbursts up to redshift 7. The kinematics of the ionised gas in the E galaxy agrees with the known[$\ion${C}{II}]gas kinematics, indicating a physical association between the ionised gas and cold dust/gas clumps. The situation in the W galaxy is more complex as it appears to be a velocity offset of about +700 km s$^{-1}$in the Pa$\alpha$relative to the[$\ion${C}{II}]emitting gas.The nature of this offset, and its reality, is not fully established and requires further investigation. The observed properties of SPT0311-58 such as the clumpy distribution at sub(kpc) scales and the very high average extinction are similar to those observed in low- and intermediate-$z$luminous (E galaxy) and ultraluminous (W galaxy) infrared galaxies, even though SPT0311-58 is observed only$\sim$800 Myr after the Big Bang. Massive, heavily obscured, clumpy starburst systems like SPT0311-58 likely represent the early phases in the formation of massive high-redshift bulge/spheroids and luminous quasars. MIRI and JWST will be able to explore for the first time the rest-frame near-infrared stellar and ionised gas structure of these galaxies, even during the Epoch of Reionization as demonstrated by this study.

</div>

<div id="div_fig1">

<img src="tmp_2301.02313/./plot_apertures.png" alt="Fig6" width="100%"/>

**Figure 6. -** Spatial distribution of ALMA and MIRI clumps. From left to right, ALMA rest-frame 160$\mu$m continuum, ALMA [$\ion${C}{II}]158 $\mu$m, MIRI/F1000W and WFC3/F160W images of SPT0311-58. MIRI and HST images have been subtracted from the lensing galaxy (see $\S$\ref{sec:3.1.}). White dashed circles display the apertures used to extract the photometry centred on ALMA continuum clumps and defined by \citet{Spilker+22}. Cyan circles mark the clumps derived in this work from the F1000W image. (*fig:im_and_apertures*)

</div>
<div id="div_fig2">

<img src="tmp_2301.02313/./plot_MRSapert.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{Left:} MIRI F1000W image of the SPT0311-58 system after removal of the lensing galaxy. Cyan circles mark the position of the individual F1000W clumps, while white circles shows the MRS apertures used for the extraction of the Pa $\alpha$ spectra (see Sect. \ref{sec:3.3.}). \textit{Right}: RGB image where red, green and blue colours shows the ALMA $\CII$ , MIRIM F1000W and WPFC3 F160W lens-subtracted images, representing the [$\ion${C}{II}], rest-frame 1.25$\mu$m and rest-frame 200nm continuum, respectively (see Sect. \ref{sec:2.}). (*fig:MRS_app*)

</div>
<div id="div_fig3">

<img src="tmp_2301.02313/./plot_fracgas_vs_z.png" alt="Fig1" width="100%"/>

**Figure 1. -** Gas fraction vs redshift. This figure shows the evolution of the gas mass fraction (i.e. $M_\mathrm{gas}/M_\mathrm{*}$) with the redshift. Black and red triangles represent the values derived for the E and W SPT0311-058 clumps, respectively, presented in Table \ref{tab:Mass_CII_reg}. Red and black empty squares display the $M_\mathrm{gas}/M_\mathrm{*}$ ratio obtained using the gas mass derived by \citet{Marrone+18} and the stellar masses computed in this work via SED fitting (see Sec. \ref{sec:4.1.}) for the E and W galaxies, respectively. For comparison, galaxies at intermediate redshifts ($z\sim$1$-$3) from the ASPECS 1.2mm sample presented in \citep{Aravena+20} are included. The higher redshift range ($z\sim$4.4$-$6) is covered with the ALPINE sample, for which we calculate the gas mass following the $L_\mathrm{CII}-M_\mathrm{gas}$ relation introduced by \citet{Zanella+18}. The black continuum, dashed and dashed–dotted lines show the gas fraction evolution following \citet{Liu+19} for a galaxy with a stellar mass of $M_*$=5$\times$10$^{10}$ M$_\odot$ and a SFR that is 1, 3 and 10 times the one expected for the SFR main sequence, respectively. (*fig:frac_gas*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\au}{au}$
$\newcommand{\juliet}{\texttt{juliet}}$
$\newcommand{\serval}{\texttt{serval}}$
$\newcommand{\raccoon}{\texttt{raccoon}}$
$\newcommand{\exostriker}{\texttt{Exo-Striker}}$
$\newcommand{\aliasfinder}{\texttt{AliasFinder}}$
$\newcommand{\tess}{TESS}$
$\newcommand{\gaia}{\textit{Gaia}}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\wolf}{Wolf~1069}$
$\newcommand{\inst}[1]{\unskip^{\instrefs{#1}}}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\mearth}{M_\oplus}$
$\newcommand{\rearth}{R_\oplus}$
$\newcommand{\prot}{169.3^{+3.7}_{-3.6}}$
$\newcommand{\scsep}{\newcommand{\}{scsep}{,}}$
$\newcommand{\}{scsep}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\au$}{au}$
$\newcommand{$\juliet$}{\texttt{juliet}}$
$\newcommand{$\serval$}{\texttt{serval}}$
$\newcommand{$\raccoon$}{\texttt{raccoon}}$
$\newcommand{$\exostriker$}{\texttt{Exo-Striker}}$
$\newcommand{$\aliasfinder$}{\texttt{AliasFinder}}$
$\newcommand{$\tess$}{TESS}$
$\newcommand{$\gaia$}{\textit{Gaia}}$
$\newcommand{$\jwst$}{JWST}$
$\newcommand{$\wolf$}{Wolf~1069}$
$\newcommand{$\inst$}[1]{\unskip^{$\inst$refs{#1}}}$
$\newcommand{$\ms$}{m s^{-1}}$
$\newcommand{$\mearth$}{M_\oplus}$
$\newcommand{$\rearth$}{R_\oplus}$
$\newcommand{$\prot$}{169.3^{+3.7}_{-3.6}}$
$\newcommand{\scsep}{\newcommand{\}{scsep}{,}}$
$\newcommand{\}{scsep}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.02477-b31b1b.svg)](https://arxiv.org/abs/2301.02477)<mark>Appeared on: 2023-01-09</mark> - _26 pages, 15 figures_

</div>
<div id="authors">

<mark><mark>D. Kossakowski</mark></mark>, et al. -- incl., <mark><mark>M. Kürster</mark></mark>, <mark><mark>T. Trifonov</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>R. Burn</mark></mark>, <mark><mark>E. Herrero</mark></mark>, <mark><mark>A. Pavlov</mark></mark>, <mark><mark>M. Pérez-Torres</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present the discovery of an Earth-mass planet ($M_\textnormal{b}\sin i = 1.36\pm0.21 M_\oplus$) on a\num{15.6}d orbit of a relatively nearby ($d \sim$9.6 pc) and low-mass ($0.167 \pm 0.011 M_\odot$) M5.0 V star,$\wolf$. Sitting at a separation of$0.0672\pm0.0014$\auaway from the host star puts$\wolf$b in the habitable zone (HZ), receiving an incident flux of$S = 0.652\pm 0.029 S_\oplus$.The planetary signal was detected using telluric-corrected radial-velocity (RV) data from the CARMENES spectrograph, amounting to a total of 262 spectroscopic observations covering almost four years. There are additional long-period signals in the RVs, one of which we attribute to the stellar rotation period. This is possible thanks to our photometric analysis including new, well-sampled monitoring campaigns undergone with the OSN and TJO facilities that supplement archival photometry (i.e., from MEarth and SuperWASP), and this yielded an updated rotational period range of$P_\textnormal{rot} = 150-170$d, with a likely value at$\prot$d. The stellar activity indicators provided by the CARMENES spectra likewise demonstrate evidence for the slow rotation period, though not as accurately due to possible factors such as signal aliasing or spot evolution.Our detectability limits indicate that additional planets more massive than one Earth mass with orbital periods of less than 10 days can be ruled out, suggesting that perhaps$\wolf$b had a violent formation history. This planet is also the sixth closest Earth-mass planet situated in the conservative HZ, after$\object${Proxima Centauri}b,$\object${GJ 1061}d,$\object${Teegarden's Star}c, and$\object${GJ 1002}b and c. Despite not transiting,$\wolf$b is nonetheless a very promising target for future three-dimensional climate models to investigate various habitability cases as well as for sub-$\ms$RV campaigns to search for potential inner sub-Earth-mass planets in order to test planet formation theories.

</div>

<div id="div_fig1">

<img src="tmp_2301.02477/./20221205_wolf1069_periodogram_carmvis_rv_tellcorr_2cols_equalzoom.png" alt="Fig10" width="100%"/>

**Figure 10. -** GLS periodograms for the RVs of $\wolf$ after sequentially subtracting out the most prominent signals. The horizontal dashed, dot-dashed, and dotted lines represent the 10 \%, 1 \%, and 0.1 \% FAP levels (from bottom to top). There were no significant signals with periods shorter than 10 d other than the aliases due to the daily sampling. The right panel is a closer zoom-in of the left panel to highlight the longer-period signals.
    The 15.6 d planetary signal is illustrated with a vertical blue solid line, and the 90.3 d signal and its alias due to the 270-d sampling with a green solid and dashed line, respectively. The range for the photometric rotation period is shaded in orange, where the stellar rotation period within the RVs is marked with an orange solid line.
    The component of residual telluric contamination at 388 d and its alias due to the 365-d sampling period are also represented with a vertical magenta solid and dashed line, respectively.
    Panel (a): the window function of the data set. Panel (b): no signal fitted, solely the original RVs with an offset and jitter term. Panel (c): residuals after subtracting the 15.6 d signal. Panel (d): residuals after subtracting a simultaneous model fit of two sinusoids at 15.6 d and 90.3 d. Panel (e): residuals after subtracting a simultaneous model fit of three sinusoids at 15.6 d, 90.3 d, and 165 d. Panel (f): residuals after subtracting the final model choice including 1 Keplerian at 15.6 d (further described in Sect. \ref{sec:modelcomparison}).
     (*fig:rvperiodogram*)

</div>
<div id="div_fig2">

<img src="tmp_2301.02477/./20221205_wolf1069_periodogram_carmvis_tellcorr_2121_bluered_2cols_zoom_martinfap.png" alt="Fig1" width="100%"/>

**Figure 1. -** GLS periodograms of the telluric-only (nonTC subtracted from TC; \textit{top}), nonTC (\textit{middle}) and TC (\textit{bottom}) RVs. The red and blue colors represent the "blue" and "red" subsets within the VIS channel of the CARMENES instrument (Sect. \ref{sec:90dsignal}). The vertical lines match those in Fig. \ref{fig:rvperiodogram} for consistency and the horizontal lines are identical in the bottom panels but differ slightly from the top panel.
     (*fig:redblue*)

</div>
<div id="div_fig3">

<img src="tmp_2301.02477/./rv_vs_time.png" alt="Fig7.1" width="50%"/><img src="tmp_2301.02477/./phased_rv_pl1.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** RV time series for and phase-folded RVs for $\wolf$ b
RV time series and phase-folded plots for $\wolf$ b.
\textit{Top panel:} CARMENES VIS RV measurements for $\wolf$  along with the best-fit model (dark gray line) and the stellar rotation period modeled by a dSHO-GP (orange). The light gray band indicates the 68\% confidence interval of the model. \textit{Bottom panel:} RVs phase-folded to the period of $\wolf$ b at 15.6 d ($K_{\rm b} = 1.07\pm0.17$\ms) and with the GP component subtracted out. The black circles represent the data points binned to 0.1 in phase space for visualization purposes. The bottom panel for each plot represents the residuals after subtracting out the model. There are two data points that did not fit within the boundary for visual reasons. (*fig:rv*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

158  publications in the last 7 days.
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers